<h2>Blum-Goldwasser Probabilistic Encryption</h2>

In [ ]:
import numpy 
import math 
import random
import time

In [ ]:
#Umbral a partir del cual consideramos que un numero es "grande"
umbral = 2
#Rango de valores a partir del umbral que se pueden generar
rango = 2000

p, q, a, b = generarClavePrivada(umbral, rango)

clavePrivada = [p, q, a, b]
clavePublica = p*q

print("Clave privada --> ", clavePrivada)
print("Clave publica --> ", clavePublica)

msjClaro = leerFichero("msjClaro.txt")
alf = leerFichero("alf.txt")
msjCifrado = encriptar(msjClaro, alf, clavePublica)
print(msjCifrado)

In [ ]:
def generarClavePrivada(umbral, rango):
    p=0; q=0

    while not cumpleCongruencia(p, 3, 4) or not esPrimo(p):
        p = random.randint(umbral, umbral+rango)

    while not cumpleCongruencia(q, 3, 4)  or p==q or not esPrimo(q): 
        q = random.randint(umbral, umbral+rango)

    a, b = calcularAlgEuclidesExtendido(p, q)
    
    return p, q, a, b

In [ ]:
def esPrimo(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False

    return True

In [ ]:
def cumpleCongruencia(num, numCongruente, modulo):
    return (num%modulo) == (numCongruente%modulo)

In [ ]:
def calcularAlgEuclidesExtendido(p, q):
    restos = [p,q]
    listaA = [1,0]
    listaB = [0,1]
    cocientes = [0,0]

    i=2

    while(restos[i-1] > 1):
        cocientes.append(restos[i-2]//restos[i-1])
        restos.append(restos[i-2]%restos[i-1])
        listaA.append(listaA[i-2] - cocientes[i] * listaA[i-1])
        listaB.append(listaB[i-2] - cocientes[i] * listaB[i-1])
        i += 1

    if restos.pop()==0:
        return listaB.pop(), 0
    
    return listaA.pop(), listaB.pop()


In [ ]:
def leerFichero(ruta):
    fichero = open(ruta, "r", encoding="utf8")
    texto = fichero.read()
    fichero.close()
    return texto

In [ ]:
def encriptar(msjClaro, alf, n):
    k = math.floor(math.log(n))
    #h es la longitud de los bloques en los que se dividira el mensaje binario
    h = math.floor(math.log(k))
    x = []
    c = []

    msjBinario = strToBinary(msjClaro, alf)
    t = len(msjBinario)

    x.append(inicializarX0(n))
    for i in range(1, t):
        x.append(pow(x[i-1], 2, n))
        p = decimalToBinary(x[i], 0)
        p = p[len(p)-h:]
        #TODOOOO No se si esta bien porque pi son los h ultimos bits de x que son varios y mi es 1 solo, entonces no se si se puede hacer un XOR
        #de por ejemplo '101001' y '1'
        c.append(int(msjBinario[i-1],2)^int(p,2))
    
    c.append(pow(x[len(x)-1], 2, n))
    return c

In [ ]:
def strToBinary(cadena, alf):
    #numero minimo de digitos binarios para representar todas las letras del alfabeto
    longBinaria = math.ceil(math.log(len(alf),2)) 
    cadenaBinaria = ''

    for caracter in cadena:
        cadenaBinaria += decimalToBinary(alf.find(caracter), longBinaria)

    return cadenaBinaria



In [ ]:
def decimalToBinary(caracterNumerico, longBinaria):
    bloqueBinario = bin(caracterNumerico)[2:]

    if(longBinaria==0):
        longBinaria = len(bloqueBinario)

    #Rellena con 0s a la izquierda 
    while(len(bloqueBinario)!=longBinaria):
        bloqueBinario = '0' + bloqueBinario
    
    return bloqueBinario


In [ ]:
def inicializarX0(n):
    r = random.randint(1, n-1)
    while not tieneInverso(r, n):
        r = random.randint(1, n-1)
    
    return pow(r, 2, n)

In [ ]:
def tieneInverso(numero, modulo):
    
    try:
        pow(numero,-1,modulo)
        return True
    except:
        return False
        
    return False